In [ ]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import os, sys
from tqdm import tqdm

sys.path.insert(0, 'src')
from utils import remove_and_create_dir, ensure_dir
from preprocess_data import preprocess_csvs

### Process a few data files

In [ ]:
data_root_dir =  os.path.join('data') # TODO: replace with your root if necessary
csv_file_names = ['rsim.csv']
csv_paths = [os.path.join(data_root_dir, csv_file_name) for csv_file_name in csv_file_names]
save_dir = os.path.join('data', 'processed_data')

preprocess_csvs(
    csv_paths, 
    verbose=True,
    csv_save_dir=save_dir)

### Process a LOT of data files organized in directories

In [ ]:
data_root_dir =  os.path.join('data', 'raw_data') # TODO: replace with your root if necessary
# csv_file_names = ['rsim.csv', 'test.csv', 'train.csv']
sub_dirs = ['itr{}'.format(i) for i in range(1,6)]
csv_file_names = [
    'sch_FIFO-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_DDR-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_WFQ-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_SP-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_FIFO-tgen_OnOff-n_flows_100-n_ports_4.csv',
    'sch_FIFO-tgen_Poisson-n_flows_20-n_ports_4.csv',
    'sch_FIFO-tgen_Poisson-n_flows_60-n_ports_4.csv',
    'sch_FIFO-tgen_Poisson-n_flows_100-n_ports_8.csv',
    'sch_FIFO-tgen_Poisson-n_flows_100-n_ports_16.csv',
]
# print(sub_dirs)

for sub_dir in sub_dirs:
    csv_paths = []
    for csv_file_name in csv_file_names:
        csv_paths.append(os.path.join(data_root_dir, sub_dir, csv_file_name))

    # csv_paths = [os.path.join(data_root_dir, csv_file_name) for csv_file_name in csv_file_names]
    save_dir = os.path.join('data', 'processed_data', sub_dir)
    ensure_dir(save_dir)
    # print(len(csv_paths))
    preprocess_csvs(
        csv_paths, 
        verbose=False,
        csv_save_dir=save_dir)

### Process DQN fattree data files to train on

In [ ]:
data_root_dir =  os.path.join('data', 'dqn_data', 'data', 'fattree16', ) # TODO: replace with your root if necessary
# csv_file_names = ['rsim.csv', 'test.csv', 'train.csv']
sub_dirs = ['map', 'onoff', 'poisson']
csv_file_names = [
    'rsim{}.csv'.format(i) for i in range(1,6)
]
# print(sub_dirs)

for sub_dir in sub_dirs:
    csv_paths = []
    for csv_file_name in csv_file_names:
        csv_paths.append(os.path.join(data_root_dir, sub_dir, csv_file_name))
    print(csv_paths)
    # csv_paths = [os.path.join(data_root_dir, csv_file_name) for csv_file_name in csv_file_names]
    save_dir = os.path.join('data', 'processed_data', 'dqn_fattree16', sub_dir)
    ensure_dir(save_dir)
    # print(len(csv_paths))
    preprocess_csvs(
        csv_paths, 
        verbose=False,
        csv_save_dir=save_dir)